## Creating a New AutoCNet Server Project

In [1]:
%env autocnet_config=/home/jlaura/autocnet_projects/test_spatiallite.yml

env: autocnet_config=/home/jlaura/autocnet_projects/test_spatiallite.yml


In [2]:
import autocnet_server
from autocnet_server.db.model import Base

from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
from sqlalchemy_utils import database_exists, create_database

In [3]:
autocnet_server.config

{'cluster': {'cluster_log_dir': '/home/jlaura/logs',
  'cluster_submission': 'slurm',
  'extractor_memory': 8192,
  'maxfailures': 3,
  'processing_memory': 8192,
  'queue': 'shortall',
  'tmp_scratch_dir': '/scratch/jlaura'},
 'database': {'database_host': 'sqlite:///gis.db',
  'database_name': 'jaydemo',
  'database_password': 'jlaura',
  'database_username': 'jlaura',
  'pgbouncer_port': 0,
  'timeout': 500},
 'developer': {'autocnet_path': '/home/jlaura/autocnet',
  'autocnet_server_path': '/home/jlaura/anaconda3/envs/ct/lib/python3.6/site-packages/autocnet_server',
  'plio_path': '/home/jlaura/anaconda3/envs/ct/lib/python3.6/site-packages/plio/'},
 'directories': {'vrt_dir': '/sratch/jlaura/ctx_demo/vrt'},
 'python': {'env_name': 'ct',
  'pybin': '/home/jlaura/anaconda3/envs/ct/bin/python',
  'pyroot': '/home/jlaura/anaconda3/envs/ct'},
 'redis': {'basename': 'jdemo',
  'completed_queue': 'jdemo:done',
  'host': 'autocnet.wr.usgs.gov',
  'port': '8084',
  'processing_queue': 'jdem

This is the code that creates the database and associated tables for an AutoCNet project. No database names or login information is specified in this file.  This is because everything is being stored in a `YAML` configuration file.

In [4]:
config = autocnet_server.config

engine = create_engine(config['database']['database_host'])
create_database(engine.url)
Base.metadata.bind = engine
Base.metadata.create_all(engine)

CompileError: (in table 'edges', column 'ring'): Compiler <sqlalchemy.dialects.sqlite.base.SQLiteTypeCompiler object at 0x2aea8db76278> can't render element of type <class 'sqlalchemy.sql.sqltypes.ARRAY'>

In [4]:
config = autocnet_server.config
engine = create_engine('postgresql://{}:{}@{}:{}/{}'.format(config['database']['database_username'],
                                                            config['database']['database_password'],
                                                            config['database']['database_host'],
                                                            config['database']['pgbouncer_port'],
                                                            config['database']['database_name']))
if not database_exists(engine.url):
    create_database(engine.url, template='template_postgis')

# Create the tables
Base.metadata.bind = engine
Base.metadata.create_all(engine)

ValueError: invalid literal for int() with base 10: ''